In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from string import punctuation
import numpy as np
import pandas as pd
import pickle, re
from typing import List, Dict, Any
import datetime

In [2]:
STORES_DICT = {
    "the_bar": "The Bar",
    "casadabebida": "Casa da Bebida"
}

In [3]:
def get_meta_info(product: str, store: str) -> Dict:
  sku = None
  brand = None
  description = None
  availability = None
  price_currency = None
  price_amount = None
  url = None

  if all_products[product][store] is not None:
    url = all_products[product][store]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    if store=="the_bar":
      sku = soup.find("meta", property="product:sku")
      brand = soup.find("meta", property="product:brand")
      price_currency = soup.find("meta", property="product:price:currency")
      availability = soup.find("meta", property="product:availability")
      price_amount = soup.find("meta", property="product:price:amount")

      sku = sku['content'] if sku else None
      brand = brand['content'] if brand else None
      price_currency = price_currency['content'] if price_currency else None
      availability = availability['content'] if availability else None
      price_amount = float(price_amount['content']) if price_amount else None

    elif store=="casadabebida":
      sku = soup.find(itemprop="sku").get_text()
      brand = soup.find(itemprop="brand").get_text().strip()

      description = soup.find("meta", property="og:title")
      description = description['content'] if description else None

      price_currency = soup.find(itemprop="priceCurrency").get("content")
      availability = soup.find(itemprop="availability").get("content").split("/")[-1]
      price_amount = float(soup.find(itemprop="price").get_text())

  infos = {
      "sku": sku, 
      "marca": brand,
      "descricao": description,
      "disponibilidade": availability,
      "preco_moeda": price_currency,
      "preco_valor": price_amount,
      "loja": STORES_DICT[store],
      "data": datetime.datetime.now().isoformat(),
      "url": url
    }
  return infos

In [ ]:
all_products = {
    "singleton_12y": {
        "the_bar": "https://br.thebar.com/whisky-singleton-of-dufftown-12-anos--750ml-gre30284/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/singleton-12-anos-of-glen-ord-700-ml/",
    },
    "glenkinchie_12y": {
        "the_bar": "https://br.thebar.com/whisky-glenkichie-12-anos--750ml-gre30252/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenkinchie-12-anos-750-ml/"
    },
    "glenfiddich_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/single-malt/whisky-glenfiddich-12-anos/"
    },
    "talisker_10y":  {
        "the_bar": "https://br.thebar.com/whisky-talisker-10-anos-750ml-gre30290/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/talisker-10-anos-750-ml/"
    },
    "johnnie_green": {
        "the_bar": "https://br.thebar.com/whisky-johnnie-walker-green-label--750ml-gre30263/p",
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-johnnie-walker-green-label/"
    },
    "glenlivet_15y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/glenlivet-15-anos-750-ml/"
    },
    "laphroaig_10y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-laphroaig-10-anos/"
    },
    "macallan_sherryoak_12y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/macallan-sherry-oak-12-anos-700-ml/"
    },
    "macallan_18y": {
        "the_bar": None,
        "casadabebida": "https://www.casadabebida.com.br/whisky/whisky-the-macallan-18-anos/"
    }
  }

In [ ]:
get_meta_info("glenkinchie_12y", "casadabebida")

In [ ]:
data = [get_meta_info(x,'casadabebida') for x in all_products.keys()]

In [ ]:
dataframe = pd.DataFrame(data=data, columns = ["marca", "descricao", "disponibilidade", "loja", "sku","preco_moeda", "preco_valor", "data", "url"])
dataframe

In [ ]:
dataframe.sort_values(by="preco_valor", ascending=True, inplace=True)

In [ ]:
dataframe=dataframe.reset_index(drop=True)

In [ ]:
dataframe

In [ ]:
dataframe = dataframe.set_index('data')

In [ ]:
dataframe

In [ ]:
dataframe.index

In [5]:
old_df = pd.read_csv("../output/dataframe_casadabebida.csv", index_col=0, parse_dates=True, infer_datetime_format=True)

In [6]:
old_df.tail()

,marca,descricao,disponibilidade,loja,sku,preco_moeda,preco_valor,url
data,,,,,,,,
2022-04-01 11:56:16.482897,Johnnie Walker,Whisky Johnnie Walker Green Label 750 ml,InStock,Casa da Bebida,15,BRL,379.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01 11:56:17.251146,Glenlivet,Whisky Glenlivet 15 anos 750 ml,InStock,Casa da Bebida,1139,BRL,379.91,https://www.casadabebida.com.br/whisky/glenliv...
2022-04-01 11:56:17.946285,Laphroaig,Whisky Laphroaig 10 Anos 750 ml,InStock,Casa da Bebida,750,BRL,398.91,https://www.casadabebida.com.br/whisky/whisky-...
2022-04-01 11:56:18.780784,Macallan,Whisky Macallan Sherry Oak 12 anos 700 ml,InStock,Casa da Bebida,2744,BRL,906.21,https://www.casadabebida.com.br/whisky/macalla...
2022-04-01 11:56:19.503811,Macallan,Whisky The Macallan 18 anos - Double Cask 700 ml,InStock,Casa da Bebida,226,BRL,3061.76,https://www.casadabebida.com.br/whisky/whisky-...


In [7]:
import altair as alt


In [8]:
# import altair as alt
# from vega_datasets import data
# #alt.renderers.enable('altair_saver', fmts=['vega-lite', 'png'])

# temps = data.seattle_temps()
# temps.head()

In [9]:
#emps.date[0]

In [10]:
# temps = temps[temps.date < '2010-01-15']


# alt.Chart(temps).mark_line().encode(
#     x='date:T',
#     y='temp:Q'
# )

In [11]:
# alt.Chart(plot_df).mark_line().encode(
#     x='data:T',
#     y='preco_valor:Q'
# )

In [12]:
plot_df = old_df.reset_index()
plot_df = plot_df[["data", "marca", "preco_valor"]]
plot_df

,data,marca,preco_valor
0,2022-04-01 11:56:13.450845,Singleton,189.91
1,2022-04-01 11:56:14.943618,Glenfiddich,284.91
2,2022-04-01 11:56:14.191140,Glenkinchie,351.41
3,2022-04-01 11:56:15.728990,Talisker,379.91
4,2022-04-01 11:56:16.482897,Johnnie Walker,379.91
5,2022-04-01 11:56:17.251146,Glenlivet,379.91
6,2022-04-01 11:56:17.946285,Laphroaig,398.91
7,2022-04-01 11:56:18.780784,Macallan,906.21
8,2022-04-01 11:56:19.503811,Macallan,3061.76


In [13]:
plot_df.data[0]

Timestamp('2022-04-01 11:56:13.450845')

In [15]:
chart = alt.Chart(plot_df).mark_line().encode(
    x='data:T',
    y='preco_valor:Q',
    color='marca'
).properties(
    title="Whisky Price Trends",
    width=840,
    height=320,
)
chart

alt.Chart(...)

In [17]:
from altair_saver import save
alt.renderers.enable('altair_saver', fmts=['vega-lite', 'png'])

RendererRegistry.enable('altair_saver')

In [18]:
save(chart, "chart_1.png") 

/Users/rsilvei/node_modules/vega-label/build/vega-label.js:593
      let textWidth = d.textWidth ?? 0,
                                   ^

SyntaxError: Unexpected token '?'
    at wrapSafe (internal/modules/cjs/loader.js:915:16)
    at Module._compile (internal/modules/cjs/loader.js:963:27)
    at Object.Module._extensions..js (internal/modules/cjs/loader.js:1027:10)
    at Module.load (internal/modules/cjs/loader.js:863:32)
    at Function.Module._load (internal/modules/cjs/loader.js:708:14)
    at Module.require (internal/modules/cjs/loader.js:887:19)
    at require (internal/modules/cjs/helpers.js:74:18)
    at Object.<anonymous> (/Users/rsilvei/node_modules/vega/build/vega-node.js:13:13)
    at Module._compile (internal/modules/cjs/loader.js:999:30)
    at Object.Module._extensions..js (internal/modules/cjs/loader.js:1027:10)


CalledProcessError: Command '['/Users/rsilvei/node_modules/.bin/vl2vg']' returned non-zero exit status 1.

In [ ]:
chart.save('chart_whisky.png')